In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import threading
import math
import pandas as pd
from scipy.optimize import minimize
import asyncio
import datetime
import hashlib
import os
import json

In [3]:
def change(filename):
        f = open(f"{filename}.txt", "r")
        out = open(f"{filename}.csv", "w")
        for line in f:
                change = line.replace("\t", ",")
                out.write(change)

In [4]:
change("Demand")

In [5]:
def read_file(filename):
    with open(filename,"r") as f:
        df = pd.read_csv(f,header=None)
        return df.to_numpy()

In [6]:
shops = read_file("Demand.csv")
values = np.array(shops[:, 3])
points = np.array(shops[:, 1:3])
x_range = [1, 9]
y_range = [0, 9]
n = len(values)
cost = 40
prohibited_rectangles = [((3.5, 5.2), (4.8, 6.2)), ((2.7, 3.3),  (3.0, 3.8)), ((3.2, 3.0),  (3.8, 3.4)), ((4.7, 4.2), (5.0, 4.5)), ((5.9, 2.9),  (6.2, 3.8))]
prohibited_circles = [((6.25, 7.45), 0.75)]

In [15]:
def le(x, y):
    epsilon = 1e-9
    return (x - y) >= epsilon
def is_in_prohibited_area(point, prohibited_rectangles, prohibited_circles):
    x, y = point
    for (x1, y1), (x2, y2) in prohibited_rectangles:
        if le(x, x1) and le(x2, x) and le(y, y1) and le(y2, y):
            return True
    for (xc, yc), radius in prohibited_circles:
        if le(radius, np.linalg.norm([x - xc, y - yc])):
            return True
    return False

# Function to perform constrained KMeans clustering
def constrained_kmeans(points, values, max_sum, m):
    clusters = KMeans(n_clusters=m).fit(points)
    labels = clusters.labels_
    
    while True:
        cluster_sums = [np.sum(values[labels == i]) for i in range(m)]
        if all(sum_val < max_sum for sum_val in cluster_sums):
            break
        for i, sum_val in enumerate(cluster_sums):
            if sum_val >= max_sum:
                indices = np.where(labels == i)[0]
                np.random.shuffle(indices)
                for idx in indices:
                    for j in range(m):
                        if j != i and np.sum(values[labels == j]) + values[idx] < max_sum:
                            labels[idx] = j
                            break
    return labels

# Modified function to find the geometric median avoiding prohibited areas
def geometric_median_avoid_prohibited(X, weights, prohibited_rectangles, prohibited_circles, eps=1e-5):
    def objective(y):
        penalty = 0
        if is_in_prohibited_area(y, prohibited_rectangles, prohibited_circles):
            penalty = 1e6  # Large penalty for points inside prohibited areas
        distances = cdist(X, [y])
        weighted_sum = np.sum(distances * weights[:, np.newaxis])
        return weighted_sum + penalty
    
    y = np.mean(X, axis=0)
    result = minimize(objective, y, method='SLSQP')
    return result.x

# Function to calculate weighted sum of distances
def calculate_weighted_sum_of_distances(medians, points, values, labels):
    weighted_sums_of_distances = []
    for i, median in enumerate(medians):
        if median is not None:
            cluster_points = points[labels == i]
            cluster_values = values[labels == i]
            distances = np.linalg.norm(cluster_points - median, axis=1)
            weighted_distances = distances * cluster_values
            weighted_sum_of_distances = np.sum(weighted_distances)
            weighted_sums_of_distances.append(weighted_sum_of_distances)
        else:
            weighted_sums_of_distances.append(None)
    return weighted_sums_of_distances

# Function to plot clusters
def plot_clusters(points, labels, medians, filename='clusters_with_medians.png'):
    plt.ioff()
    plt.figure(figsize=(15, 15))
    colors = plt.get_cmap('tab10', len(medians))
    for i in range(len(medians)):
        cluster_points = points[labels == i]
        if len(cluster_points) > 0:
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], color=colors(i), label=f'Cluster {i}')
            if medians[i] is not None:
                plt.scatter(medians[i][0], medians[i][1], color='black', marker='x', s=100, label=f'Median {i}')
                for point in cluster_points:
                    plt.plot([medians[i][0], point[0]], [medians[i][1], point[1]], color=colors(i), linestyle='-')
    # Plot prohibited rectangles
    for (x1, y1), (x2, y2) in prohibited_rectangles:
        plt.fill([x1, x2, x2, x1], [y1, y1, y2, y2], color='Aquamarine', alpha=1)
    # Plot prohibited circles
    for (xc, yc), radius in prohibited_circles:
        circle = plt.Circle((xc, yc), radius, color='Aquamarine', alpha=1)
        plt.gca().add_patch(circle)
    plt.xlim(x_range)
    plt.ylim(y_range)
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Clusters and Geometric Medians with Connecting Lines')
    plt.legend()
    plt.grid(True)
    plt.savefig(filename, dpi=300)
    plt.close()


# Main function to run the tasks
def run_task(idx, points, values, startm, endm, max_sum, try_num):
    file_name = f"Task{idx}.txt"
    f = open(file_name, "w")
    for m in range(startm, endm):
        for runtime in range(try_num):
            labels = constrained_kmeans(points, values, max_sum, m)
            geometric_medians = []
            ans = 0
            check = True
            for i in range(m):
                cluster_values = values[labels == i]
                cluster_points = points[labels == i]
                if len(cluster_points) > 0:
                    geometric_median_point = geometric_median_avoid_prohibited(cluster_points, cluster_values, prohibited_rectangles, prohibited_circles)
                    geometric_medians.append(geometric_median_point)
                else:
                    geometric_medians.append(None)
                    check = False
                    break
            if check == False:
                print(f"Runtime {runtime} of {m} cannot find geometric medians for all clusters")
                continue
            
            weighted_sums_of_distances = calculate_weighted_sum_of_distances(geometric_medians, points, values, labels)

            for i, sum_dist in enumerate(weighted_sums_of_distances):
                if sum_dist is not None:
                    ans += sum_dist
            ans += cost * m
            check = True
            # print(f"Minimum cost is {ans}")
            for i, median in enumerate(geometric_medians):
                if median is not None and is_in_prohibited_area(median, prohibited_rectangles, prohibited_circles):
                    check = False
                    print(f"Fuck YourSelf {runtime} of {m}")
            if check:
                plot_clusters(points, labels, geometric_medians, filename=f'Task{idx}_m{m}_runtime{runtime}.png')
                f.write(f"{m} \n")
                for i, median in enumerate(geometric_medians):
                    f.write(f"{median[0]} {median[1]} \n")
                f.write(f"{ans} \n")
                    
            
            


# Thread management to test all values of m from 1 to n with 10 tries each
if __name__ == "__main__":
    max_sum = 1000
    
    threads = []
    # for m in range(1, n+1):
    #     for try_num in range(1, 11):
    #         t = threading.Thread(target=run_task, args=(points, values, startm, endm, max_sum, try_num))
    #         threads.append(t)
    #         t.start()
    
    # for t in threads:
    #     t.join()
    run_task(0, points, values, 10, 11, max_sum, 10)
    print("Completed all tasks.")

Runtime 1 of 10 cannot find geometric medians for all clusters
Completed all tasks.
